In [1]:
#!/usr/bin/env python
import numpy as np
import math as m
import pylab as pl
from matplotlib import colors
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter
from colormaps import spectral
from matplotlib.colors import ColorConverter
from scipy import special
from matplotlib.backends.backend_pdf import PdfPages

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# Font properties
# Note: TNR required for bolding due to font manager bug
# Math font for compact, times-like scientific notation
# Exit out of notebook before changing
times_font=True
if times_font:
    font_bold = fm.FontProperties(family='Times New Roman', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Times', style='normal', size=14, weight='normal', stretch='normal')
    font_name ='Times'
    plt.rcParams["mathtext.fontset"] = "stix"#"stixsans"
else:
    font_name ='Dejavu Sans'
    font_bold = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='bold', stretch='normal')
    font      = fm.FontProperties(family='Dejavu Sans', style='normal', size=14, weight='normal', stretch='normal')


In [5]:
## Epoch of Reionisation redshifts
# publication metadata
        
#*****Paciga et al., 2013. DOI: 10.1093/mnras/stt753
#name, mK^2, k [h Mpc^-1), redshift]
paciga_2013 = ('Paciga, 2013', [6.15E4], [.5], [8.6])

#*****Dillon et al., 2014. DOI: 10.1103/PhysRevD.89.023002
dillon_2014 = ('Dillon, 2014', 
              [2.6E7,1.16E6,8.64E5,6.7E5,1.3E6,1.28E7,5.26E7,5.67E8,4.58E6,2.93E8,6.92E8],
              [.058,.06,.063,.065,.0678,.0712,.0715,.149,.15,.15,.089],
              [11.68,10.868,10.153,9.518,8.444,7.985,7.57,7.1896,6.84,6.52,6.23])

#*****Dillon et al., 2015. DOI: 10.1103/PhysRevD.91.123011
dillon_2015 = ('Dillon, 2015',
              [3.8E4,3.69E4,4.67E4],
              [.18,.18,.16],
              [6.4,6.8,7.25])

#*****Beardsley et al., 2016. DOI: 10.3847/1538-4357/833/1/102
beardsley_2016 = ('Beardsley, 2016',
                 [2.70E4,3.02E4,3.22E4,3.2E4,2.6E4,2.5E4],
                 [.27,.24,.24,.16,.14,.14],
                 [7.1,6.8,6.5,7.1,6.8,6.5])

#*****Patil et al., 2017. DOI: 10.3847/1538-4357/aa63e7
patil_2017 = ('Patil, 2017',
             [131.5**2.,242.1**2.,220.9**2.,337.4**2.,407.7**2.,  86.4**2.,144.2**2.,184.7**2.,296.1**2.,342.0**2.,  79.6**2.,108.8**2.,148.6**2.,224.0**2.,366.1**2.],
             [.053,.067,.083,.103,.128,  .053,.067,.083,.103,.128,  .053,.067,.083,.103,.128],
             [8.3,8.3,8.3,8.3,8.3,  9.15,9.15,9.15,9.15,9.15,  10.1,10.1,10.1,10.1,10.1])

#*****Kolopanis et al, 2019. DOI: 10.3847/1538-4357/ab3e3a
kolopanis_2019 = ('Kolopanis, 2019',
                 [8.09e4,1.28e5,3.61e4,1.46e5,3.8e6,2.41e6],
                 [0.39,0.53,0.31,0.36,0.334,0.33],
                 [7.49,8.13,8.37,8.68,9.93,10.88])

#*****Barry et al., 2019. DOI: 10.3847/1538-4357/ab40a8
full_hist_barry_2019 = ('Barry, 2019',
                        [0,9.22E3,3.89E3,6.32E3,9.61E3,1.56E4,2.07E4,8.32E4,0,0,7.09E4,3.2E4,4.43E4,6.38E4,
                        1.11E5,2.62E5,2.28E5,4.25E4,1.22E5,0,0,1.35E6,7.01E4, 5.91E4, 3.24E5,5.32E5,2.42E5,7.41E4,8.57E4,1.13E5,4.65E5,
                        0,0,0,0,0,2.32E6,7.22E5,5.48E5,6.58E5,4.67E5,6.16E5,1.48E6,2.15E6,1.73E6],
                        [0.145,0.174,0.203,0.232,0.261,0.290,0.319,0.349,0.3776,0.4937,0.523,0.552,0.581,0.610,0.639,
                        0.668,0.697,0.726,0.755,0.784,0.900,0.929,0.958,0.987,1.017,1.046,1.075,1.104,1.133,1.162,1.191,1.220,1.249,1.278,
                        1.307,1.336,1.365,1.394,1.423,1.452,1.481,1.510,1.539,1.568,1.597],
                        [7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7])
barry_2019 = ('Barry, 2019',
              [3273.5603],
              [0.20330593],
              [7])

#*****Li et al., 2019. DOI: 10.3847/1538-4357/ab55e4
li_2019 = ('Li, 2019',
          [9.28E3,5.59E3,2.39E3],
          [.25,.29,.59],
          [7.1,6.8,6.5])

#***** Metens et al., 2020
mertens_2020 = ('Mertens, 2020',
               [72.86**2],
               [.075],
               [9.1])

#***** Trott et al., 2020. DOI: 10.1093/mnras/staa414
trott_2020 = ('Trott, 2020',
             [43.1**2,60.1**2,77.7**2,154.2**2,167.7**2,249.6**2],
             [0.142,0.142,0.142,0.142,0.142,0.142],
             [6.5,6.8,7.1,7.8,8.2,8.7])

#***** Rahimi et al., 2021. DOI: 10.1093/mnras/stab2918
rahimi_2021 = ('Rahimi, 2021',
              [5.44E3, 9.28E3, 1.52E4],
              [0.13,0.13,0.13],
              [6.5,6.8,7.1])

#***** HERA et al., 2022. DOI: 10.3847/1538-4357/ac1c78
hera_2021 = ('HERA, 2022',
            [(30.76)**2.,(95.74)**2.],
            [0.192,0.256],
            [7.9,10.4])

#***** HERA et al., 2023
hera_2023 = ('HERA, 2023',
            [(30.76)**2.,(95.74)**2.],
            [0.192,0.256],
            [7.9,10.4])

#***** Kolopanis et al. 2023. DOI: 10.1093/mnras/stad845
kolopanis_2023 = ('Kolopanis, 2023',
                  [4.58E3],
                  [.19],
                  [7.14])

#***** Wilensky et al. in review

In [ ]:
def plot_2d:
    
        min_k = .01
        max_k = 1.1
        min_d = 10**0
        max_d = 10**6
        fontsize = 26
        
    
        #Plot defaults
        fig = plt.figure(figsize=(21,21))
        plt.loglog()
        plt.tick_params(direction='in', color='k', which='both', top=True, bottom=True, 
                        left=True, right=True, labelsize=fontsize, width=2, length=6)
        plt.xlim(min_k,max_k)
        plt.ylim(min_d,max_d)
        plt.ylabel('$\Delta^2$ (mK$^2$)', fontsize=fontsize)
        plt.xlabel('$k$ (h Mpc$^{-1}$)', fontsize=fontsize)